# 텍스트와 바이트

## 문자 문제
* 문자열
    * 문자의 열
* 문자
    * ```python2```: ```ascii``` ([출처](https://libsora.so/posts/python-hangul/))
    * ```python3```: ```uicode```
* 인코딩과 디코딩
![chap04-1_encoding-decoding](../images/chap04-1_encoding-decoding.png)

## 바이트에 대한 기본 지식
* bytes
    * python 3에서 소개
    * 불변형
    * python 2.6의 bytes형과 다름
* bytearray
    * python 2.6 추가
    * 가변형
* 특징
    * 0 ~ 255 사이 정수를 항목으로 가짐
    * 슬라이싱하면 동일한 이진 시퀀스 만들어짐
    * ```format()```, ```format_map()``` 제외 ```str``` 제공 메소드 지원
    * ```str```에 없는 ``` fromhex()``` (공백으로 구분된 16진수 쌍을 파싱해 이진 시퀀스 생성) 제공
* 출력
    * 출력 가능 아스키 문자 (공백에서 물결표)까지 아스키 문자로 출력
    * 탭, 개행문자, 캐리지 리턴, 백슬래시는 이스케이프 시퀀스로 출력
    * 그외 값 16진수 이스케이프 시퀀스로 출력
* 생성자의 인자로 가능한 것들
    * str과 encoding 키워드 인수
    * 0 ~ 255 사이 값을 갖는 반복 가능형
    * 하나의 정수 (이를 사용할 시 해당 정수만큼의 널 바이트로 초기화된 이진 시퀀스 생성,python 3.6에서 제거)
    * 버퍼 프로토콜을 구현하는 객체
        * 메모리 공유하는 ```memeoryview```와 달리 바이트를 복사
 * ```struct``` 모듈
     * 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석
     * 튜플을 패킹된 바이트로 변환하는 함수 제공
     * ```bytes, bytearray, memoryview``` 객체와 함께 사용
     * format 형식은 [여기](https://docs.python.org/3/library/struct.html)를 참고

In [1]:
# Python 3.5.2
# bytes
print('-----bytes-----')
cafe = bytes('café', encoding='utf-8')
print('bytes로 변환: {}'.format(cafe))
print('cafe[0]: {}'.format(cafe[0]))
print('cafe[0]: {}'.format(bytes([99])))
print('cafe[:1]: {}'.format(cafe[:1]), end='\n\n')

# bytearray
print('-----bytearray-----')
cafe_arr = bytes(cafe)
print('bytearray로 변환: {}'.format(cafe_arr))
print('cafe_arr[-1:]: {}'.format(cafe_arr[-1:]))

-----bytes-----
bytes로 변환: b'caf\xc3\xa9'
cafe[0]: 99
cafe[0]: b'c'
cafe[:1]: b'c'

-----bytearray-----
bytearray로 변환: b'caf\xc3\xa9'
cafe_arr[-1:]: b'\xa9'


In [24]:
import struct


fmt = '<3s3sHH'
with open('../images/chap04-1_struct-example.png', 'rb') as f:
    img = memoryview(f.read())
    
header = img[:10]
print('bytes: {}'.format(bytes(header)))
print('struct unpack: {}'.format(struct.unpack(fmt, header)))

del header
del img

bytes: b'\x89PNG\r\n\x1a\n\x00\x00'
struct unpack: (b'\x89PN', b'G\r\n', 2586, 0)


## 기본 인코더/디코더
* 코덱
    * 인코더/디코더
    * ```open(), str.encode(), bytes.decode()``` 등 호출 시 ```encoding``` 인수에 전달해 사용
* 알아두면 유용한 코덱 (한국 기준)
    * ascii
        * 문자를 7bit로 표현 ([출처](https://ko.wikipedia.org/wiki/ASCII))
        * 한국어, 중국어 등 표현하기 부족
    * utf-8
        * 문자를 8bit로 표현 ([출처](https://excelsior-cjh.tistory.com/117))
        * 웹에서 8bit 인코딩을 하기 위해 가장 널리 사용되는 방식
        * ascii 코드와 하위 호환
    * cp949 ([출처](https://ko.wikipedia.org/wiki/%EC%BD%94%EB%93%9C_%ED%8E%98%EC%9D%B4%EC%A7%80_949))
        * 마이크로소프트사 한글 문자 표현 방식
        * ```euc-kr```의 확장
        * 하위 호환성 있음
    * euc-kr ([출처](https://ko.wikipedia.org/wiki/EUC-KR))
        * 한글 완성형 인코딩
        * 문자를 8bit로 표현

## 인코딩/디코딩 문제 이해하기
* ```UnicodeEncodeError```
    * 텍스트를 바이트로 변환할 시 문자가 대상 인코딩에 정의되지 않았으면 별도의 처리기 지정하지 않은 한 해당 에러 발생
    * ```codecs.register_error()``` 메서드로 처리기명과 처리 함수명 전달해 에러 처리기 등록 가능
* ```UnicodeDecodeError```
    * 이진 시퀀스를 텍스트로 변환할 시 정당한 문자로 변환할 수 없으면 발생
* ```SyntaxError```
    * 인코딩 선언 없이 기본 인코딩 방식이 아닌 코덱으로 인코딩된 모듈 로딩 시 발생
* 바이트 시퀀스의 인코딩 방식을 알아내는 방법은 없기 때문에 별도로 인코딩 정보 가져올 것
    * ```b'\x00``` 많이 나타나면 16이나 32비트 인코딩일 가능성 틈
    * ```b'\x20\x00'``` 많이 나타나면 UTF-16LE 인코딩 공백 문자일 가능성 큼
    * ```Chardet``` 패지키로 문자 인코딩 감지할 수 있음
    * 리눅스에서는 ```file```명령어로 인코딩 확인하고, ```iconv```로 코덱 변경 ([참고](https://genesis8.tistory.com/37]))
* BOM (Byte Order Mark, [출처](https://genesis8.tistory.com/37))
    * c.f. Javascript BOM (Browser Object Model)
    * 엔디언 (endian)
        * 컴퓨터 메모리와 같은 1차원 공간에 여러 개의 연속된 대상 배열하는 방법
        * 바이트 순서 (Byte Order)
    * 리틀 엔디언 (little endian)
        * 작은 단위의 바이트가 앞에 옴
        * ```b'\xff\xfe'``` 인텔 CPU 리틀엔디언
    * 빅 엔디언 (big endian)
        * 큰 단위의 바이트가 앞에 옴
    * ```ZERO WIDTH NO-BREAK SPACE```
        * 화면에 출력되지 않음
        * 코덱에 따라 빅엔디언 또는 리틀엔디언 명시
    * 한 바이트 이상을 워드로 사용하는 UTF-16, UTF-32에 영향
    * 윈도우 어플리케이션은 UTF-8 인코딩하면서 파일에 BOM 붙임

## 텍스트 파일 다루기
![unicode sandwitch](../images/chap04-1_unicode-sandwitch.png)
* 기본 인코딩(```locale.getpreferredencoding(do_setlocale=True)```)에 의존하면 문제가 발생할 수 있음
    * 유니코드 샌드위치 모델을 따르고 프로그램 안에서 늘 인코딩 명시할 것을 권장
* 래스터 이미지 등 이진 파일을 열 때만 이진 모드 사용 권장
* 인코딩 방식 분석 시에도 ```Chardet``` 모듈 사용 권장

## 보충
* 래스터 이미지 (Raster Image)
    * 비트맵 (Bitmap, 이진 숫자 배열표)
    * 이미지 저장 파일 포맷 또는 메모리 저장 방식
    * 화소 단위
    * e.g. JPEG, GIF, PNG
    ![Raster Image](../images/chap04-1_raster-image.png)
* c.f. 벡터 그래픽스 (Vector Graphics, e.g. svg)

* binary sequence v.s. byte sequence ([출처](https://stackoverflow.com/questions/25722611/difference-between-storing-images-in-byte-array-and-binary-blob-and-which-one))
    * bit: binary digit의 약자로, 가장 작은 저장 단위. 0 또는 1의 값을 가짐.
    * nibble: 1바이트의 절반으로 보통 4비트를 가리키는 컴퓨터 환경의 용어. 옥텟의 절반. 니블이 4비트가 포함되어 있어 16 이라는 값이 있다고 할 수 있으므로 니블은 하나의 십육진수와 일치. hex digit.
    * byte: (자주 사용하지 않는 nibble을 제외) 두 번째로 작은 저장 단위. 8 bits.
    * binary: 0 또는 1의 값을 지님.
    * BLOB: 데이터베이스에서 바이너리 데이터를 저장할 때 사용. 

![jjal](../images/chap04-1_jjal.png)